### Initialization

In [1]:
import glob
import json
import re
from nltk import tokenize
import textdistance
import random
import string
import os
import numpy
import pandas
import fasttext
import gensim
import conllu
import editdistance
import udapi

In [2]:
from utils.text_extraction import *
from utils.document_processing import *
from utils.subject_extraction import *
from utils.similarity import *
from utils.context_extraction import *

In [3]:
import psycopg2
psycopg2_conn = psycopg2.connect(dbname='public_contracts', user='postgres', password='admin', host='localhost', port='5432')

In [4]:
from ufal.udpipe import *
# load model from the given file;
# if the file does not exist, expect a Segmentation fault
udpipe_model = Model.load("udpipe/udpipe-ud-2.4-190531/czech-pdt-ud-2.4-190531.udpipe")

In [5]:
# create a UDPipe processing pipeline with the loaded model,
# with "horizontal" input (a sentence with space-separated tokens),
# default setting for tagger and parser,
# and CoNLL-U output
udp_pipeline = Pipeline(udpipe_model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

### Documents 

In [6]:
from utils.document_processing import *

In [7]:
loader = DatabaseDocumentLoader(psycopg2_conn)
loader.load_documents()
documents = loader.prepare_documents()

Running query: select * from document where processed=True
Preparing total 1008 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%


In [8]:
vzdirs = '../test-data/*/*/*/*'
ignore_contracts = ['2096','sportovni-areal-zs-slatina-1-etapa_25534',
                    'revitalizace-lokality-spaliste-k-u-stare-mesto-u-uherskeho-hradiste-akceptacni-cislo-12136894-cz-1-02-4-2-00-12-16399-dodatecne-prace_9496',
                    '35226', 'kompostujeme-v-obci-sebranice-dodavka-stepkovace-a-komposteru', 'zvysovani-povedomi-verejnosti-o-biodiverzite-luk-a-pastvin',
                    'nakup-noveho-uzitkoveho-elektromobilu-typu-bev-vozidlo-s-bateriovym-pohonem']

ref_loader = ReferenceDocumentLoader(vzdirs)
# ref_loader.extract_text_from_documents(0,1024)
ref_loader.load_documents_from_extracts(ignore_contracts)
ref_documents = ref_loader.prepare_documents()

Loading total 316 extracts
Preparing total 316 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 59.0%
Progress: 69.0%
Progress: 79.0%
Progress: 89.0%
Progress: 99.0%


In [9]:
ignore_mask = count_occurence_vector('qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNMáčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ0123456789')
ignore_mask[0]=0
matcher = DocumentMatcher(ref_documents, documents)
matcher.count_most_similar_documents(ignore_mask=ignore_mask)
matcher.aggregate_documents()
df_contract = matcher.filter_aggregated()

Counting most similar documents for total 316 reference documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 59.0%
Progress: 69.0%
Progress: 79.0%
Progress: 89.0%
Progress: 99.0%


### Subject context

In [10]:
from utils.context_extraction import *

In [11]:
contracts = []
for contr_name, docs in df_contract.groupby('contr_name'):
    contr_text = ''
    contr_ids = []
    for index, doc in docs.iterrows():
        doc_name = doc['doc_name']
        contr_text += '======================='+doc_name+'=======================\n'
        contr_text += doc['doc_text']
        contr_ids.append(doc['contr_id'])
    contr_id = get_most_frequent(contr_ids)
    contr = {'id': contr_id, 'contr_name': contr_name, 'text': contr_text}
    contracts.append(contr)

In [12]:
keywords = {
    'Předmět smlouvy':10,
    'Předmět díla':10,
    'Předmět plnění':10,
    'Předmět veřejné zakázky':10,
    'Vymezení předmětu':10,
    'Vymezení plnění': 10,
    'Název veřejné zakázky': 3,
    'Veřejná zakázka':1,
    'Veřejné zakázce':1,
    'Předmět': 1
   }

extractor = AdvancedSubjectContextExtractor(keywords=keywords, subj_range=2000)
for contr in contracts:
    contr_text = contr['text']
    contr['subj_context'] = extractor.get_subject_context(contr_text)

In [13]:
df_contracts = pandas.DataFrame(contracts)
# df_contracts

### Subject extraction

In [14]:
from utils.subject_extraction import *

In [15]:
from utils.subject_context_preprocessing import *

In [16]:
def count_line_lengths_percentile(text, q=95):
    lines = [line for line in text.split('\n')]
    line_lengths = [len(line) for line in lines]
    return numpy.percentile(line_lengths, q)

In [17]:
pandas.DataFrame([count_line_lengths_percentile(text) for text in df_contracts['subj_context']])

,0
0,122.45
1,95.10
2,431.80
3,545.40
4,92.00
5,96.00
6,222.00
7,57.40
8,54.00
9,169.05


In [18]:
def count_longest_lines_lengths_var(text):
    lines = [line for line in text.split('\n')]
    line_lengths = [len(line) for line in lines]
    sorted_lengths = numpy.sort(numpy.array(line_lengths))[::-1]
    num = int(len(sorted_lengths)/20)+1
    top_n = sorted_lengths[:num]
    var_of_top_n = numpy.var(top_n)
    return var_of_top_n

In [19]:
pandas.DataFrame([count_longest_lines_lengths_var(text) for text in df_contracts['subj_context']])

,0
0,0.000000
1,1.555556
2,187062.500000
3,20224.888889
4,0.222222
5,4.250000
6,0.000000
7,11388.337950
8,0.000000
9,0.000000


### EvaluationMachine

In [20]:
class EvaluationMachine:
    
    _core = None
    _subjects = None
    _ref_subjects = None
    _similarity_matrix = None
    
    def __init__(self, core=None):
        self._core = core
    
    def evaluateSubjects(self, subjects, ref_subjects):
        self._subjects = [' '.join(s) if isinstance(s, list) else s for s in subjects]
        self._ref_subjects = [' '.join(s) if isinstance(s, list) else s for s in ref_subjects]
        return self.evaluate()
    
    def evaluate(self):
        self.compute_matrix()
        return self._similarity_matrix
        
    def compute_matrix(self):
        sl = len(self._subjects)
        rl = len(self._ref_subjects)
        result_matrix = numpy.zeros((sl, rl))
        for i in range(sl):
            for j in range(rl):
                d = self._core.compute(self._subjects[i], self._ref_subjects[j])
                result_matrix[i][j] = d
        self._similarity_matrix = result_matrix
        
    def display(self, with_headers=False):
        if with_headers:
            df = pandas.DataFrame(self._similarity_matrix, columns=self._ref_subjects, index=self._subjects)
        else:
            df = pandas.DataFrame(self._similarity_matrix)
        display(df)
        
    
class RandomEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(RandomSimilarityMachine())
        
class JaccardEvaluationMachine(EvaluationMachine):

    def __init__(self, core=None):
        super().__init__(JaccardSimilarityMachine())
        
class FastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(FastTextSimilarityMachine(core))
        
class GensimFastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(GensimFastTextSimilarityMachine(core))

In [21]:
class SubjectContextCopyExtractor(SubjectExtractor):
    
    def __init__(self, param=None):
        pass
        
    def extract(self, text):
        return text

### Similarity functions

In [22]:
from utils.similarity import *

### Measurements

In [23]:
vzdirs = '../test-data/*/*/*'
def save_subjects(df_contracts, column, vzdirs='../test-data/*/*/*'):
    dirs = [path for path in glob.glob(vzdirs) if 'test_src' not in path]
    for path in dirs:
        contr_name = '/'.join(path.split('\\')[1:4])
        rows=df_contracts[df_contracts['contr_name']==contr_name]
        if len(rows)>0:
            print('saving ' + contr_name)
            with open(path+'/_ref.json', encoding='utf-8') as f:
                data = f.read()
                ref_dict = json.loads(data)
            ref_dict[column]=rows.iloc[0][column] if len(rows)>0 else None
            with open(path+'/_ref.json', 'w', encoding='utf-8') as f:
                json.dump(ref_dict, f, ensure_ascii=False, indent=4)

In [24]:
def validate_subjects(df_contracts, column, vzdirs='../test-data/*/*/*', similarity_machine=JaccardSimilarityMachine()):
    ref_column = column+'_ref'
    df_contracts['valid_score']=0
    df_contracts[ref_column]=None
    ref_paths = [path for path in glob.glob(vzdirs) if 'test_src' not in path]
    for path in ref_paths:
        contr_name = '/'.join(path.split('\\')[1:4])
        rows = df_contracts[df_contracts.contr_name==contr_name]
        if len(rows)>0:
            row = rows.iloc[0]
            ref_value = None
            with open(path+'/_ref.json', 'r', encoding='utf8') as f:
                ref = json.load(f)
                if column in ref:
                    ref_value = ref[column]
            valid_score = similarity_machine.compute(row[column], ref_value)
            df_contracts.loc[row.name, 'valid_score'] = valid_score
            df_contracts.loc[row.name, ref_column] = ref_value
    return df_contracts

In [25]:
from udapi.core.document import Document

def create_conllu_sentence_list(decomposition):
    doc = Document()
    doc.from_conllu_string(decomposition)
    return [b.trees[0] for b in doc.bundles]

class SubjectSentenceEvaluationMachine:
    
    _df_contracts = None
    _subj_context_preprocessor = None
    
    def __init__(self, df_contracts, subj_context_preprocessor=SubjectContextPreprocessor):
        self._df_contracts = df_contracts
        self._subj_context_extractor = subj_context_preprocessor
        
    def preprocess(self):
        self._df_contracts['filtered_context'] = self._df_contracts['subj_context'].apply(\
            lambda text: self._subj_context_extractor.process(text))
        self._df_contracts['subj_context_decomposition'] = self._df_contracts['filtered_context'].apply(\
            lambda text: udp_pipeline.process(text))
        return self._df_contracts
    
    def process(self):
        self._df_contracts = validate_subjects(self._df_contracts, 'subj_context_decomposition')
        return self._df_contracts

    def evaluate(self):
        return self._df_contracts['valid_score'].mean()*100
    
    def evaluateSentenceNum(self):
        self._df_contracts['sent_cnt_diff'] = \
            self._df_contracts['subj_context_decomposition'].apply(lambda decomposition: len(create_conllu_sentence_list(decomposition))) \
            - self._df_contracts['subj_context_decomposition_ref'].apply(lambda decomposition: len(create_conllu_sentence_list(decomposition)))
        return self._df_contracts['sent_cnt_diff'].abs().mean()

In [158]:
%%time

# df_slice = df_sorted_contracts[:10].copy()
evaluator = SubjectSentenceEvaluationMachine(df_contracts, subj_context_preprocessor=SubjectContextPreprocessor())
evaluator.preprocess()
evaluator.process()
display(evaluator.evaluate())
display(evaluator.evaluateSentenceNum())

98.53576949626995

0.075

Wall time: 31.7 s


In [80]:
evaluator._df_contracts

,id,contr_name,text,subj_context,filtered_context,subj_context_decomposition,valid_score,subj_context_decomposition_ref,sent_cnt_diff
0,71,e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef...,=======================2017126-115451.pdf=====...,PŘEDMĚT SMLOUVY\nZhotovitel se zavazuje k prov...,PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pro...,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0.998570,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0
1,72,e-zakazky/73b0be6b-6b49-4ab4-a6a0-508b83a305e1...,=======================Formulář - Informace o ...,PŘEDMĚT DÍLA\n\n1.1. Předmětem díla je provede...,PŘEDMĚT DÍLA.\n\nPředmětem díla je provedení s...,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0.944041,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0
2,74,e-zakazky/c265fdf8-044f-44e5-b36e-df4ca53c6179...,=======================Dodatečné informace 01....,Předmět smlouvy\n1. Zhotovitel se touto smlouv...,Předmět smlouvy.\nZhotovitel se touto smlouvou...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.524780,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,7
3,61,eagri/mze/contract_display_6220,=======================01_výzva k podání nabíd...,Předmět smlouvy\n\nPředmětem smlouvy je proved...,Předmět smlouvy.\n\nPředmětem smlouvy je prove...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.999849,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0
4,64,ezak/mpsv/P15V00000447,=======================AKRIS_příloha-01_Kvalif...,Předmět veřejné zakázky a sjednaná cena ve sml...,Předmět veřejné zakázky a sjednaná cena ve sml...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.988906,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,1
5,66,ezak/mpsv/P16V00000187,=======================Příloha č. 1 SOD_Projek...,"Předmět smlouvy, předmět díla\n\nPředmětem tét...","Předmět smlouvy, předmět díla.\n\nPředmětem té...",# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.999678,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0
6,67,ezak/mpsv/P16V00000240,=======================P1_Výkaz výměr (slepý)....,Předmět Smlouvy\n2.1. Zhotovitel se zavazuje p...,Předmět Smlouvy.\nZhotovitel se zavazuje prové...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.995816,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0
7,103,nipez/mo/P18V00009525,=======================VZOR_smlouvy.doc=======...,Předmět smlouvy\n\n2.1. \nPředmětem smlouvy je...,Předmět smlouvy.\n\nPředmětem smlouvy je závaz...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.553194,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,3
8,207,nipez/mo/P18V00012708,=======================Oznámení o výběru dodav...,Název veřejné zakázky: Sponkovače a spojovací ...,Název veřejné zakázky:. Sponkovače a spojovací...,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0.944828,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0
9,397,nipez/mo/P18V00019354,=======================PSP s přílohami.zip====...,Předmět smlouvy\n\n\n1. Poskytovatel se zavazu...,Předmět smlouvy.\n\nPoskytovatel se zavazuje d...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.996865,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0


### Playground

In [28]:
class StringFinder:
    
    _pattern = None
    _lower = None
    _show_prec_range = None
    _show_next_range = None
    
    def __init__(self, keyword, lower=True, show_range=30):
        self._pattern = keyword
        self._lower = lower
        self._show_prec_range = show_range[0] if isinstance(show_range, tuple) else show_range
        self._show_next_range = show_range[1] if isinstance(show_range, tuple) else show_range
    
    def process(self, text):
        occurrences = find_all_occurrences_in_string(self._pattern, text, self._lower)
        findings = {}
        for occ in occurrences:
            findings[occ] = text[max(0, occ-self._show_prec_range): min(occ+self._show_next_range, len(text))]
        return findings
    
# class RegexFinder:
    
#     _pattern = None
#     _lower = None
    
#     def __init__(self, pattern, match='exact'):
#         self._pattern = pattern
#         self._match = match
    
#     def process(self, text):
        

class SearchEngine:
    
    _finders = None
    _findings = None

    def __init__(self, by_string=None, by_regex=None, by_conllu=None, match='exact', show_range=30):
        self._finders = []
        self._findings = {}
        for keyword in by_string:
            lower = False
            if match=='lower':
                lower = True
            self._finders.append(StringFinder(keyword, lower, show_range))
    
    def preprocess(self, data):
        data_collection = []
        if isinstance(data, str):
            data_collection.append(data)
        if isinstance(data, list) or \
           isinstance(data, pandas.core.series.Series):
            data_collection.extend([str(x) for x in data])
        return data_collection
    
    def process(self, data):
        texts = self.preprocess(data)
        for i, text in enumerate(texts):
            findings = {}
            for finder in self._finders:
                finding = finder.process(text)
                if finding:
                    findings[finder._pattern] = finding
            if findings:
                self._findings[i] = findings
        return self._findings

In [29]:
from udapi.block.util.filter import Filter
from udapi.core.document import Document

class ConlluWordOccurrenceSentenceFilter:
    _keywords = None

    def __init__(self, keywords=['cena', 'hodnota']):
        self._keywords = keywords

    def process(self, tokenlists):
        filtered_tokenlists = conllu.models.TokenList([])
        for tokenlist in tokenlists:
            if any(len(tokenlist.filter(lemma=keyword)) != 0 for keyword in self._keywords):
                continue
            filtered_tokenlists.append(tokenlist)
        return filtered_tokenlists


class UdapiTransformer:
    
    @classmethod
    def _iter_sentences(cls, document):
        sentences = []
        for b in document.bundles:
            if not b.trees:
                continue
            sentences.append(b.trees[0])
        return sentences

class UdapiWordOccurrenceSentenceFilter(UdapiTransformer):
    _keywords = None
    _udapi_filters = None

    def __init__(self, keywords=['cena', 'hodnota'], udapi_filters=None):
        self._keywords = keywords
        self._udapi_filters = udapi_filters \
            if udapi_filters is not None else \
                [Filter(delete_tree_if_node='node.lemma=="'+keyword+'"') for keyword in self._keywords]

    def process(self, conllu_document):
        for udapi_filter in self._udapi_filters:
            udapi_filter.process_document(conllu_document)
        return conllu_document
    
class UdapiWordOccurrencePartSentenceFilter(UdapiTransformer):
    _keywords = None
    _udapi_filters = None

    def __init__(self, keywords=['cena', 'hodnota'], udapi_filters=None):
        self._keywords = keywords
        self._udapi_filters = udapi_filters \
            if udapi_filters is not None else \
                [Filter(delete_subtree='node.lemma=="'+keyword+'"') for keyword in self._keywords]

    def process(self, conllu_document):
        for udapi_filter in self._udapi_filters:
            udapi_filter.process_document(conllu_document)
        return conllu_document
    
def get_preceding_node(origin, feature, value, max_depth=100):
    node = origin
    depth = 0
    while depth < max_depth:
        node = node.parent
        if not node:
            break
        depth += 1
        attrs = node.get_attrs([feature])
        if attrs:
            attr = attrs[0]
            if attr == value:
                return node
    return None
    
class SubmitterPartSentenceFilter(UdapiTransformer):
    
    _submitter_keywords = None
    _suplier_keywords = None
    
    def __init__(self, submitter_keywords=['objednatel', 'zadavatel', 'kupující'],
                 suplier_keywords=['dodavatel', 'prodávající', 'zhotovitel', 'uchazeč']):
        self._submitter_keywords = submitter_keywords
        self._suplier_keywords = suplier_keywords
        
    def process(self, document):
        for tree in self._iter_sentences(document):
            for n in tree.descendants:
                if n.lemma.lower() in ['objednatel', 'kupující', 'zadavatel'] and n.feats['Case'] == 'Nom':
                    p = get_preceding_node(n, 'upos', 'VERB')
                    if not p:
                        p = n
                    for n2 in p.descendants:
                        if n2.lemma.lower() in ['dodavatel', 'prodávající', 'zhotovitel', 'uchazeč'] \
                                and n2.feats['Case'] == 'Nom':
                            p2 = get_preceding_node(n2, 'upos', 'VERB')
                            if not p2:
                                p2 = n2
                            p2.parent = p.parent
                    p.remove()
                    tree.text = tree.compute_text()
        return document
       
class EmptyBundlesFilter:
    
    def process(self, document):
        bundles = []
        for b in document.bundles:
            if not b.trees:
                continue
            if not b.trees[0]:
                continue
            if not b.trees[0].descendants:
                continue
            bundles.append(b)
        document.bundles = bundles
        return document

class ConlluSubjectContextPreprocessor:
    _transformers = None

    def __init__(self, transformers=None):
        self._transformers = transformers \
            if transformers is not None else \
            [
                UdapiWordOccurrenceSentenceFilter(keywords=['cena', 'hodnota', 'DPH']),
                EmptyBundlesFilter(),
            ]

    def process(self, conllu_document):
        for transformer in self._transformers:
            conllu_document = transformer.process(conllu_document)
        return conllu_document

In [30]:
# TODO
# vyfiltrovat ceny (cena, hodnota, kč) 17, 23, 27,29
# vyfiltrovat věty: objednatel, kupující (i uprostřed věty), náklad a nebezpečí, technická specifikace, příloha, místo, doba

In [157]:
save_subjects(df_sorted_contracts, 'subj_context_decomposition')

saving e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef/P17V00000001
saving e-zakazky/73b0be6b-6b49-4ab4-a6a0-508b83a305e1/P12V00000002
saving e-zakazky/c265fdf8-044f-44e5-b36e-df4ca53c6179/P17V00000001
saving eagri/mze/contract_display_6220
saving ezak/mpsv/P15V00000447
saving ezak/mpsv/P16V00000187
saving ezak/mpsv/P16V00000240
saving nipez/mo/P18V00009525
saving nipez/mo/P18V00012708
saving nipez/mo/P18V00019354
saving nipez/mo/P18V00022561
saving nipez/mo/P19V00003599
saving nipez/mpsv/P18V00010416
saving nipez/mvcr/P18V00009631
saving nipez/mvcr/P18V00015153
saving nipez/mzv/P18V00000055
saving nipez/mzv/P18V00016340
saving nipez/mzv/P18V00017124
saving nipez/mzv/P18V00026759
saving nipez/mzv/P19V00001827
saving stavebnionline/121/7941
saving tenderarena/agrozet/102972
saving tenderarena/Comgate/201480
saving tenderarena/JihoceskyKraj/255714
saving tenderarena/Libereckykraj/219512
saving tenderarena/mo/171826
saving tenderarena/msp/85449
saving vhodne-uverejneni/00254592/oprava-cast

In [ ]:
validate_subjects(df_contracts, 'subj_context_decomposition')

In [31]:
df_sorted_contracts = df_contracts.copy()
df_sorted_contracts['context_len'] = df_sorted_contracts.subj_context.str.len()
df_sorted_contracts = df_sorted_contracts.sort_values('context_len').reset_index()
df_sorted_contracts

,index,id,contr_name,text,subj_context,filtered_context,subj_context_decomposition,valid_score,subj_context_decomposition_ref,sent_cnt_diff,context_len
0,13,733,nipez/mvcr/P18V00009631,=======================017 Oznámení o výběru A...,veřejné zakázce\n\n„Náboje 308 Winchester SWISS“,"veřejné zakázce.\n\n""Náboje 308 Winchester SWI...",# newdoc\n# newpar\n# sent_id = 1\n# text = ve...,0.945017,# newdoc\n# newpar\n# sent_id = 1\n# text = ve...,0,46
1,8,207,nipez/mo/P18V00012708,=======================Oznámení o výběru dodav...,Název veřejné zakázky: Sponkovače a spojovací ...,Název veřejné zakázky:. Sponkovače a spojovací...,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0.944828,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0,54
2,15,771,nipez/mzv/P18V00000055,=======================Plovoucí vinylová podla...,Název veřejné zakázky:\n\tPlovoucí vinylová po...,Název veřejné zakázky:.\n\tPlovoucí vinylová p...,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0.793503,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,1,67
3,14,755,nipez/mvcr/P18V00015153,=======================#KS DNS ICT 11-2018-III...,Předmět plnění\n1. Předmětem této smlouvy je d...,Předmět plnění.\nPředmětem této smlouvy je dod...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.995984,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0,229
4,6,67,ezak/mpsv/P16V00000240,=======================P1_Výkaz výměr (slepý)....,Předmět Smlouvy\n2.1. Zhotovitel se zavazuje p...,Předmět Smlouvy.\nZhotovitel se zavazuje prové...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.995816,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0,262
5,9,397,nipez/mo/P18V00019354,=======================PSP s přílohami.zip====...,Předmět smlouvy\n\n\n1. Poskytovatel se zavazu...,Předmět smlouvy.\n\nPoskytovatel se zavazuje d...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.996865,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0,337
6,39,849,vhodne-uverejneni/zakladni-skola-velke-hamry-s...,=======================deklerace k dodatku č. ...,Název veřejné zakázky:\nZadavatel zakázky:\n\n...,Název veřejné zakázky:.\nZadavatel zakázky:.\n...,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0.958412,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0,339
7,31,831,vhodne-uverejneni/buly-arena-aquapark-kravare-...,=======================xx Písemná zpráva zadav...,předmět veřejné zakázky a cena sjednaná ve sml...,předmět veřejné zakázky a cena sjednaná ve sml...,# newdoc\n# newpar\n# sent_id = 1\n# text = př...,0.997268,# newdoc\n# newpar\n# sent_id = 1\n# text = př...,0,456
8,0,71,e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef...,=======================2017126-115451.pdf=====...,PŘEDMĚT SMLOUVY\nZhotovitel se zavazuje k prov...,PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pro...,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0.998570,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0,610
9,16,775,nipez/mzv/P18V00016340,=======================VZ-096.docx============...,Předmět veřejné zakázky\nStručný popis předmět...,Předmět veřejné zakázky.\nStručný popis předmě...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0.979726,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0,627


In [32]:
transformers = \
            [
                NumeralLinesFilter(too_many_numerals_ratio_threshold=0.5),
                TooShortLinesFilter(too_short_line_threshold=5),
                IrrelevantLinesFilter(keywords=['strana', 'stránka'], lower=True),
                IrrelevantLinesRegexFilter(patterns=[r'www', r'[\w\-\.]+@([\w\-]+\.)+[\w\-]{2,4}']),  # email
                RegexReplaceTransformer(pattern_to_transform=r'\n[ \t]*(.{0,1}[\d]+.{0,1})+[ \t]*',  # paragraph numbers
                                        result_pattern='\n'),
                BlankLinesFilter(replacement='\n', top_n_frequency=200, top_n_var_threshold=5,
                                 full_line_threshold=0.85, min_max_line_length=70),
                ReplaceMarksTransformer(marks_to_transform='„“', result_mark='"'),
                RegexReplaceTransformer(pattern_to_transform=r'([^\n])[ ]*\n', result_pattern='\g<1>.\n'),
                ReplaceMarksTransformer(marks_to_transform=[':'], result_mark=':.'),
                ReplaceMarksTransformer(marks_to_transform=['..'], result_mark='.'),
                AddLine(line='\n'),
                QuotedContractNameExtractor(name_tag='<CONTRACT_NAME>;<CONTRACT_NAME/>',
                                            false_keywords=['přílo', 'dále', 'jen'],
                                            positive_keywords=['názvem'],
                                            context_range=50, min_length=25),
                StructuredContractNameExtractor(name_tag='<CONTRACT_NAME>;<CONTRACT_NAME/>',
                                                false_keywords=['přílo', 'dále', 'jen', '"'],
                                                positive_keywords=['název'], min_length=5, delim=':.'),
                StructureItemEnumerationExtractor(item_tag='<ITEM>;<ITEM/>', enumeration_pattern=r'010(10)*',
                                                  full_line_length=150, delim=':.'),
                CharItemEnumerationExtractor(item_tag='<ITEM>;<ITEM/>',
                                             enumeration_pattern=r'1+',
                                             forbidden_chars='aábcčdďeéěfghiíjklmnňoópqrřsštťuúůvwxyýzž0123456789'),
            ]

conllu_transformers = [
    UdapiWordOccurrencePartSentenceFilter(keywords=['cena', 'hodnota', 'DPH']),
    SubmitterPartSentenceFilter(submitter_keywords=['objednatel', 'zadavatel', 'kupující'],
                                suplier_keywords=['dodavatel', 'prodávající', 'zhotovitel', 'uchazeč']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['příloha']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['náklad', 'nebezpečí','odpovědnost']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['místo', 'doba']),
    EmptyBundlesFilter(),
]
preprocessor = SubjectContextPreprocessor()
conllu_preprocessor = ConlluSubjectContextPreprocessor(transformers=conllu_transformers)

In [33]:
# 18, 24, 34, 37, 38
# 21, 32, 39
# fp 26, 27, 33, 

In [156]:
index = 39
text = df_sorted_contracts.subj_context.iloc[index]
# print(text)
transformed_text = preprocessor.process(text)
print(transformed_text)

Předmět smlouvy.
Zhotovitel se touto smlouvou zavazuje realizovat předmět veřejné zakázky "Žádovice – odkanalizování obce", kterým je dobudování kanalizace v obci Žádovice a výstavba čistírny odpadních vod včetně příslušných stavebních objektů a provozních souborů nutných pro provoz ČOV. Dále je předmětem zakázky vybudování nové kanalizace v obci Žádovice v části zvané Horní újezd, z důvodu odvedení dešťových a splaškových vod z jednotlivých rodinných domů. Navrhuje se jednotná kanalizace, která bude odvádět splaškové a dešťové vody na plánovanou ČOV. Do doby, než bude vybudována obecní ČOV, bude kanalizace odvádět jen vody dešťové. Splaškové vody budou jako doposud akumulovány v žumpách (jímkách na vyvážení) a pravidelně vyváženy k ekologické likvidaci. Dešťové vody z rodinných domů i do budoucna ale doporučujeme využívat k zalévání nebo vsakovat na vlastních pozemcích. Předmět stavebního díla, jakož i druhy, kvalita a množství výrobků a prací nezbytných k jeho realizaci, jsou vymezen

In [154]:
df_sorted_contracts.loc[index, 'subj_context_decomposition'] = udp_pipeline.process(edited_text)

In [37]:
%%time

decomposition = udp_pipeline.process(transformed_text)
doc = Document()
doc.from_conllu_string(decomposition)
doc = conllu_preprocessor.process(doc)
doc

Wall time: 210 ms


Předmět veřejné zakázky.
Stručný popis předmětu VZ:.
Disk do serveru T620(service tag :.6fdrwx1) Dell HDD 600GB SAS 15K 3.5".
Položky předmětu:.
Název položky.
Kód z NIPEZ.
Název z NIPEZ.
Kód z CPV.
Název z CPV.
Množství.
Měrná jednotka/Vlastní jednotka.
Disk do serveru T620(service tag :.6fdrwx1) Dell HDD 600GB SAS 15K 3.5".
Diskové jednotky.
Diskové jednotky. kus.
Bližší specifikace předmětu VZ:.
Název položky:.
Disk do serveru T620(service tag :.6fdrwx1) Dell HDD 600GB SAS 15K 3.5".
Stručný popis položky:.
Disk do serveru T620(service tag :.6fdrwx1) Dell HDD 600GB SAS 15K 3.5".
je:.
14 dní ode dne platnosti smlouvy.
plnění veřejné zakázky:.
sídlo zadavatele

In [35]:
transformed_texts = [preprocessor.process(t) for t in df_sorted_contracts.subj_context]

In [ ]:
se = SearchEngine(by_string=['<CONTRACT_NAME>'], match='exact', show_range=(0,150))
se.process(transformed_texts)

In [ ]:
tmp_doc = Document()
tmp_doc.from_conllu_string(decomposition)
tmp_doc

In [ ]:
tree = tmp_doc.bundles[0].trees[0]
tree

In [ ]:
node = tree.descendants[20]
node.get_attrs(['ord', 'form', 'lemma', 'upos', 'xpos', 'deprel', 'feats','misc'])

In [ ]:
nodes = tree.descendants

In [ ]:
node.get_attrs(['feats[Case]'])

In [ ]:
tree.print_subtree()

In [ ]:
tmp_doc.store_conllu('../test-data/conllu_temp_'+str(index)+'.txt')

In [ ]:
tmp_doc.store_conllu('../test-data/conllu_crit.txt')

In [ ]:
bundle = doc.bundles[8]
bundle

In [ ]:
tree = bundle.trees[0]
tree

In [ ]:
node = tree.descendants[0]
node

In [ ]:
node.lemma

In [ ]:
tree.print_subtree()

In [ ]:
WordOccurrenceSentenceFilter().process(forest)

In [ ]:
def save_connlu_to_file(connlu_data, path='../test-data/conllu_temp.txt'):
    if isinstance(connlu_data, list):
        connlu_data = '\n'.join([sentence.serialize() for sentence in connlu_data])
    if isinstance(connlu_data, conllu.models.TokenList):
        connlu_data = connlu_data.serialize()
    with open(path, 'w', encoding='utf-8') as f:
        f.write(connlu_data)

def save_text_as_connlu(text, udp_pipeline, path='../test-data/conllu_temp.txt'):
    decomposition = udp_pipeline.process(text)
    save_connlu_to_file(decomposition, path)

In [ ]:
save_connlu_to_file(forest, path='../test-data/conllu_temp_'+str(index)+'.txt')

In [ ]:
text='Předmět zakázky je automobil a předpokládaná hodnota veřejné zakázky bez DPH je 1.011.000,- Kč.'
decomposition = udp_pipeline.process(text)
tokenlist = conllu.parse(decomposition)[0]
tokenlist

In [ ]:
defects = {}

In [ ]:
defects[index]="""vyplynou v průběhu provádění díla nebo případné změny požadované objednatelem budou
řešeny následovně:
str. 2/10

a) Objednatel nebo zhotovitel předloží druhé straně "Oznámení o změně" (dále jen OZ),
ve kterém popíše požadovanou změnu."""